In [17]:
import json, os, copy, random
import numpy as np
from collections import Counter
from tqdm import tqdm
import pickle

In [4]:
dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/WebQA_0904_concat.json", "r"))

print(Counter([dataset[k]['split'] for k in dataset]))
print(Counter([dataset[k]['Qcate'] for k in dataset]))
print(Counter([dataset[k]['Qcate'] for k in dataset if dataset[k]['split'] == 'test']))
print(len(set([dataset[k]['Guid'] for k in dataset])))

Counter({'train': 36766, 'test': 7540, 'val': 4966})
Counter({'text': 24343, 'YesNo': 8255, 'Others': 6470, 'choose': 5201, 'number': 2318, 'color': 2058, 'shape': 627})
Counter({'text': 4076, 'Others': 1058, 'choose': 981, 'YesNo': 935, 'color': 228, 'number': 200, 'shape': 62})
49272


In [5]:
### Load img files' lineidx
img_lineidx = {}

# positive images for img-based queries
with open("/data/yingshac/MMMHQA/base64/gold_img.lineidx", "r") as fp:
    lineidx = [int(i.strip()) for i in fp.readlines()]
    print(len(lineidx))
    img_lineidx[0] = lineidx
# negative images for img-based queries
with open("/data/yingshac/MMMHQA/base64/neg_img.lineidx", "r") as fp:
    lineidx = [int(i.strip()) for i in fp.readlines()]
    print(len(lineidx))
    img_lineidx[1] = lineidx
# negative images for txt-based queries
with open("/data/yingshac/MMMHQA/base64/x_neg_img.lineidx", "r") as fp:
    lineidx = [int(i.strip()) for i in fp.readlines()]
    print(len(lineidx))
    img_lineidx[2] = lineidx

### img tsv files; paths
img_tsv = {0: "/data/yingshac/MMMHQA/base64/gold_img.tsv",
          1: "/data/yingshac/MMMHQA/base64/neg_img.tsv", 
          2: "/data/yingshac/MMMHQA/base64/x_neg_img.tsv"}

22265
340194
240700


In [49]:
image_ids_all = set()
for g in tqdm(dataset):
    if 'img_posFacts' in dataset[g]:
        for im in dataset[g]['img_posFacts']:
            image_ids_all.add(str(im['image_id']))
    for im in dataset[g]['img_negFacts']:
            image_ids_all.add(str(im['image_id']))
print(len(image_ids_all))

100%|██████████| 49272/49272 [00:00<00:00, 104118.74it/s]

389750


In [50]:
print(sorted(image_ids_all, key=lambda x: int(x))[-100:])

['20240481', '20240483', '20240491', '20240492', '20240493', '20240494', '20240496', '20240497', '20240498', '20240499', '20240500', '20240501', '20240502', '20240503', '20240504', '20240505', '20240506', '20240507', '20240508', '20240509', '20240510', '20240511', '20240512', '20240513', '20240514', '20240515', '20240516', '20240519', '20240522', '20240523', '20240524', '20240525', '20240533', '20240534', '20240536', '20240550', '20240551', '20240552', '20240553', '20240554', '20240555', '20240556', '20240557', '20240558', '20240569', '20240573', '20240574', '20240577', '20240581', '20240590', '20240592', '20240593', '20240594', '20240595', '20240597', '20240598', '20240599', '20240610', '20240611', '20240612', '20240613', '20240614', '20240615', '20240619', '20240620', '20240622', '20240623', '20240624', '20240625', '20240626', '20240627', '20240630', '20240632', '20240634', '20240635', '20240640', '20240641', '20240642', '20240643', '20240646', '20240650', '20240656', '20240658', '20

In [62]:
print(len(image_ids_all))
new_ids = list(range(30000000,30000000+len(image_ids_all)))
random.shuffle(new_ids)
print(len(new_ids))

389750
389750


In [63]:
image_id_map_0904 = dict(zip(image_ids_all, new_ids))
print(len(image_id_map_0904))
map_sortedby_newid = dict(sorted(image_id_map_0904.items(), key=lambda x: x[1]))
pickle.dump(map_sortedby_newid, open("image_id_map_0904.pkl", "wb"))

389750


In [69]:
map_sortedby_newid['00002098']

30123989

In [67]:
for i in list(map_sortedby_newid.keys())[-8:]:
    print(map_sortedby_newid[i])

30389742
30389743
30389744
30389745
30389746
30389747
30389748
30389749


In [73]:
### Write new tsv, combined
fp = open("/data/yingshac/MMMHQA/base64_0904/imgs.tsv", "w")
fp_lineidx = open("/data/yingshac/MMMHQA/base64_0904/imgs.lineidx", "w")
fp0 = open(img_tsv[0], "r")
fp1 = open(img_tsv[1], "r")
fp2 = open(img_tsv[2], "r")
count = 0
for i in tqdm(map_sortedby_newid):
    assert count == map_sortedby_newid[i] % 10000000
    if int(i) // 10000000 == 0:
        fp0.seek(img_lineidx[0][int(i)%10000000])
        fp_lineidx.write('{0}\n'.format(fp.tell()))
        line = [map_sortedby_newid[i], fp0.readline().strip().split('\t')[-1]]
        v = '{0}\n'.format('\t'.join(map(str, line)))
        fp.write(v)
    elif int(i) // 10000000 == 1:
        fp1.seek(img_lineidx[1][int(i)%10000000])
        fp_lineidx.write('{0}\n'.format(fp.tell()))
        line = [map_sortedby_newid[i], fp1.readline().strip().split('\t')[-1]]
        v = '{0}\n'.format('\t'.join(map(str, line)))
        fp.write(v)
    elif int(i) // 10000000 == 2:
        fp2.seek(img_lineidx[2][int(i)%10000000])
        fp_lineidx.write('{0}\n'.format(fp.tell()))
        line = [map_sortedby_newid[i], fp2.readline().strip().split('\t')[-1]]
        v = '{0}\n'.format('\t'.join(map(str, line)))
        fp.write(v)
    else: 
        print("F***")
        print(i)
    count += 1
fp.close()
fp0.close()
fp1.close()
fp2.close()
fp_lineidx.close()

100%|██████████| 389750/389750 [1:37:34<00:00, 66.57it/s]  
